In [111]:
pip install -r Project1/requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Project1/requirements.txt'


In [112]:
pip list

Package                       Version
----------------------------- ---------------
alabaster                     0.7.12
anaconda-client               1.11.2
anaconda-navigator            2.4.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
black                         22.6.0
bleac

In [113]:
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Data downloading

In [114]:
def get_data(instrument: str,
             start_date: str,
             end_date: str,
             interval: str) -> pd.DataFrame:
    """
        Fetch historical market data from Yahoo Finance for a given instrument between the provided start and end dates at the given interval.
        The function returns a cleaned DataFrame with the data, excluding any missing values.

        Parameters:
        instrument (str): The ticker symbol of the instrument e.g. 'MSFT'.
        start_date (str): The start date for the data in 'YYYY-MM-DD' format.
        end_date (str): The end date for the data in 'YYYY-MM-DD' format.
        interval (str): The time interval between data points. Valid intervals are: ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'].

        Returns:
        pd.DataFrame: A Pandas DataFrame containing the historical market data for the given instrument, with any rows containing missing values (NaN) removed.
    """
    
    data = yf.download(tickers=instrument,
                       start=start_date,
                       end=end_date,
                       interval=interval)
    
    data = data.dropna(how='any')

    return data

In [115]:
def add_vix(df: pd.DataFrame, start_date: str, end_date: str, interval: str) -> pd.DataFrame:
    """
        Add the VIX adjusted close prices as a new column to an existing DataFrame.
        
        Parameters:
        df (pd.DataFrame): The existing DataFrame containing market data with a DateTime index.
        start_date (str): The start date for fetching VIX data in 'YYYY-MM-DD' format.
        end_date (str): The end date for fetching VIX data in 'YYYY-MM-DD' format.
        interval (str): The time interval between data points for VIX.
            Valid intervals are: ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'].

        Returns:
        pd.DataFrame: The original DataFrame with an additional 'VIX' column containing adjusted close prices.
    """

    vix_data = yf.download(tickers='^VIX', start=start_date, end=end_date, interval=interval)
    vix_data = vix_data[['Adj Close']].rename(columns={'Adj Close': 'VIX'})
    df = df.merge(vix_data, how='left', left_index=True, right_index=True)
    
    return df


# Indicators

## MACD

In [116]:
def macd(data: pd.DataFrame) -> pd.DataFrame:
    """
        Calculate the MACD (Moving Average Convergence Divergence) and generate buy/sell signals.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data.

        Returns:
        pd.DataFrame: DataFrame with additional columns for 'MACD Buy Signal' and 'MACD Sell Signal', where a 1 indicates a signal and 0 means no signal.
    """

    data = data.copy()

    data['EMA12'] = data['Adj Close'].ewm(span=12, adjust=False).mean()
    data['EMA26'] = data['Adj Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA12'] - data['EMA26'] # MACD Line
    data['Signal Line'] = data['MACD'].ewm(span=9, adjust=False).mean() # Signal Line

    data['MACD Buy Signal'] = 0
    data['MACD Sell Signal'] = 0

    for i in range(1, len(data)):
        # buy signal: MACD crosses above Signal Line
        if data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] <= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Buy Signal'] = 1
        # sell signal: MACD crosses below Signal Line
        elif data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] >= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Sell Signal'] = 1

    # drop intermediate columns
    data = data.drop(columns=['EMA12', 'EMA26'])

    return data

## RSI


In [117]:
def rsi(data: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Calculate the Relative Strength Index (RSI)

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data.
    period (int): The period over which to calculate RSI, typically 14 days.

    Returns:
    pd.DataFrame: DataFrame with an additional 'RSI' column.
    """
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    return data

## ATR

In [118]:
def atr(data: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Calculate the Average True Range (ATR)

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data.
    period (int): The period over which to calculate ATR, typically 14 days.

    Returns:
    pd.DataFrame: DataFrame with an additional 'ATR' column.
    """
    high_low = data['High'] - data['Low']
    high_close = abs(data['High'] - data['Adj Close'].shift())
    low_close = abs(data['Low'] - data['Adj Close'].shift())
    
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    data['ATR'] = true_range.rolling(window=period).mean()
    
    return data

## PCA calculation

In [119]:
def apply_pca_trend_following(data: pd.DataFrame, indicators: list = ['MACD', 'RSI', 'ATR'],
                           n_components: int = 2, buy_threshold: float = 0.5, sell_threshold: float = -0.5) -> pd.DataFrame:
    """
    Apply PCA to reduce specified indicators to principal components and generate buy/sell signals.

    Parameters:
    data (pd.DataFrame): DataFrame containing calculated indicators.
    indicators (list): List of column names to include in PCA (default is ['MACD', 'RSI', 'ATR']).
    n_components (int): Number of principal components to keep (default is 2).
    buy_threshold (float): Threshold above which to generate a buy signal for PC1.
    sell_threshold (float): Threshold below which to generate a sell signal for PC1.

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'PC1', 'PC2' (if n_components=2), 'PCA Buy Signal', and 'PCA Sell Signal'.
    """
    # Ensure that required indicators are in the DataFrame
    for indicator in indicators:
        if indicator not in data.columns:
            raise ValueError(f"Missing indicator {indicator} in data. Please calculate {indicator} first.")
    
    # Standardize the data for the selected indicators
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data[indicators])

    # Apply PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data_scaled)

    # Add the principal components to the DataFrame
    for i in range(n_components):
        data[f'PC{i+1}'] = principal_components[:, i]
    
    # Generate buy/sell signals based on the first principal component (PC1)
    data['PCA Buy Signal'] = 0
    data['PCA Sell Signal'] = 0

    for i in range(1, len(data)):
        # Buy Signal
        if data['PC1'].iloc[i] > buy_threshold:
            data.loc[data.index[i], 'PCA Buy Signal'] = 1
        # Sell Signal
        elif data['PC1'].iloc[i] < sell_threshold:
            data.loc[data.index[i], 'PCA Sell Signal'] = 1

    return data

## Trend-following strategy

In [120]:
def trend_following_strategy(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate buy/sell signals based on MACD, RSI, and ATR indicators for trend-following strategy.

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data with MACD, RSI, and ATR.

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'Trend Buy Signal' and 'Trend Sell Signal'.
    """
    data = data.copy()
    
    data['Trend Buy Signal'] = 0
    data['Trend Sell Signal'] = 0

    for i in range(1, len(data)):
        # Buy signal: MACD bullish, RSI > 50, and ATR is high
        if (data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and 
            data['RSI'].iloc[i] > 50 and 
            data['ATR'].iloc[i] > data['ATR'].mean()):
            data.loc[data.index[i], 'Trend Buy Signal'] = 1
        
        # Sell signal: MACD bearish, RSI < 50, and ATR is high
        elif (data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and 
              data['RSI'].iloc[i] < 50 and 
              data['ATR'].iloc[i] > data['ATR'].mean()):
            data.loc[data.index[i], 'Trend Sell Signal'] = 1

    return data

## Trend-reversal strategy

In [121]:
# def trend_reversal_strategy(data: pd.DataFrame) -> pd.DataFrame:
#     """
#     Calculate buy/sell signals based on MACD, RSI, and VIX indicators for a trend reversal strategy.

#     Parameters:
#     data (pd.DataFrame): DataFrame containing historical market data with columns for 'MACD', 'Signal Line', 'RSI', and 'VIX'.

#     Returns:
#     pd.DataFrame: DataFrame with additional columns for 'Trend Buy Signal' and 'Trend Sell Signal'.
#     """
#     data = data.copy()
    
#     data['Trend Buy Signal'] = 0
#     data['Trend Sell Signal'] = 0

#     for i in range(1, len(data)):
#         # Buy signal: RSI < 40, MACD bullish, VIX is high
#         if (data['RSI'].iloc[i] < 40 and 
#             data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and 
#             data['VIX'].iloc[i] > data['VIX'].mean()):
#             data.loc[data.index[i], 'Trend Buy Signal'] = 1

#         # Sell signal: RSI > 60, MACD bearish, VIX is low
#         elif (data['RSI'].iloc[i] > 60 and 
#               data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and 
#               data['VIX'].iloc[i] < data['VIX'].mean()):
#             data.loc[data.index[i], 'Trend Sell Signal'] = 1

#     return data

In [122]:
def trend_reversal_strategy(data: pd.DataFrame, vix_window: int = 20) -> pd.DataFrame:
    """
    Calculate buy/sell signals based on MACD, RSI, and VIX indicators for a trend reversal strategy.

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data with columns for 'MACD', 'Signal Line', 'RSI', and 'VIX'.
    vix_window (int): Rolling window size for calculating VIX mean (default is 20).

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'Trend Buy Signal' and 'Trend Sell Signal'.
    """
    data = data.copy()

    # Calculate rolling VIX mean
    data['VIX Rolling Mean'] = data['VIX'].rolling(window=vix_window).mean()

    # Initialize signal columns
    data['Trend Buy Signal'] = 0
    data['Trend Sell Signal'] = 0

    for i in range(2, len(data)): 
        # Check for bullish divergence (buy signal)
        macd_divergence = (data['MACD'].iloc[i] > data['MACD'].iloc[i-1]) and (
            data['Adj Close'].iloc[i] < data['Adj Close'].iloc[i-1])
        if (data['RSI'].iloc[i] < 30 and
            macd_divergence and
            data['VIX'].iloc[i] > data['VIX Rolling Mean'].iloc[i]):
            data.loc[data.index[i], 'Trend Buy Signal'] = 1

        # Check for bearish divergence (sell signal)
        macd_divergence = (data['MACD'].iloc[i] < data['MACD'].iloc[i-1]) and (
            data['Adj Close'].iloc[i] > data['Adj Close'].iloc[i-1])
        if (data['RSI'].iloc[i] > 70 and
            macd_divergence and
            data['VIX'].iloc[i] < data['VIX Rolling Mean'].iloc[i]):
            data.loc[data.index[i], 'Trend Sell Signal'] = 1

    return data


In [123]:
def mean_reversion_strategy(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate buy/sell signals based on RSI, Bollinger Bands, and VIX indicators for a mean-reversion strategy.

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data with columns for RSI, Bollinger Bands, and VIX.

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'Mean Buy Signal' and 'Mean Sell Signal'.
    """
    data = data.copy()
    
    
    data['Mean Buy Signal'] = 0
    data['Mean Sell Signal'] = 0
    
    # Calculate Bollinger Bands
    data['Middle Band'] = data['Adj Close'].rolling(window=20).mean()
    data['Upper Band'] = data['Middle Band'] + 2 * data['Adj Close'].rolling(window=20).std()
    data['Lower Band'] = data['Middle Band'] - 2 * data['Adj Close'].rolling(window=20).std()

    for i in range(2, len(data)):
        # Buy signal: RSI < 30, price below lower band, VIX > 30
        if (data['RSI'].iloc[i] < 30 and 
            data['Adj Close'].iloc[i] < data['Lower Band'].iloc[i] and 
            data['VIX'].iloc[i] > 30):
            data.loc[data.index[i], 'Mean Buy Signal'] = 1

        # Sell signal: RSI > 70, price above upper band, VIX < 15
        elif (data['RSI'].iloc[i] > 70 and 
              data['Adj Close'].iloc[i] > data['Upper Band'].iloc[i] and 
              data['VIX'].iloc[i] < 15):
            data.loc[data.index[i], 'Mean Sell Signal'] = 1

    return data

In [124]:
strategies = {
    'MACD': macd,
    'PCA-trend-following' : apply_pca_trend_following,
    'Trend-Following': trend_following_strategy,
    'Trend-Reversal': trend_reversal_strategy,
    'Mean-Reversal': mean_reversion_strategy
}

In [125]:
def run_strategy(data: pd.DataFrame, strategy: str) -> pd.DataFrame:
    """
        Executes the specified trading strategy on the given data.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data on which the strategy will be applied.
                             The DataFrame must have relevant columns for the selected strategy.
        
        strategy (str): The name of the strategy to be applied.

        Returns:
        pd.DataFrame: DataFrame with the strategy applied, including any newly added columns like buy/sell signals.
    """
    
    if strategy in strategies:
        data = strategies[strategy](data=data)
    else:
        print("Invalid strategy selected!")

    return data

# Plots

## Buy and sell signal plot

In [126]:
def plot_buy_sell_signal(data: pd.DataFrame,
                         instrument: str,
                         buy_signal_column: str,
                         sell_signal_column: str,
                         title: str):
    """
        Plots a price chart with buy and sell signals marked on it.

        Parameters:
        data (pd.DataFrame): A DataFrame containing historical market data. The DataFrame must have
                            at least the 'Adj Close' column and the specified buy/sell signal columns.
        
        instrument (str): The ticker symbol or name of the instrument being plotted.

        buy_signal_column (str): The column name containing the buy signals (e.g., 'MACD Buy Signal').
                                The function will mark buy signals where the value in this column is 1.

        sell_signal_column (str): The column name containing the sell signals (e.g., 'MACD Sell Signal').
                                The function will mark sell signals where the value in this column is 1.
        
        title (str): The title of the plot, which will be displayed on the chart after instrument name.
    """
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Adj Close'],
        mode='lines',
        name='Price',
        line=dict(color='blue')
    ))

    # add buy signals to the price chart
    buy_signals = data[data[buy_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=buy_signals,
        y=data.loc[buy_signals, 'Adj Close'],
        mode='markers',
        name=buy_signal_column,
        marker=dict(color='green', symbol='triangle-up', size=10)
    ))

    # add sell signals to the price chart
    sell_signals = data[data[sell_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=sell_signals,
        y=data.loc[sell_signals, 'Adj Close'],
        mode='markers',
        name=sell_signal_column,
        marker=dict(color='red', symbol='triangle-down', size=10)
    ))

    fig.update_layout(
        title=f"{instrument} - {title}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=False,
        hovermode="x unified",
    )

    fig.show()

# Performance Evaluation

## Simulate Trading

In [127]:
def simulate_trading(data: pd.DataFrame, buy_signal_column: str, sell_signal_column: str) -> pd.DataFrame:
    """
    Simulates trading based on buy and sell signals and returns a DataFrame with trade details.

    Parameters:
    - data (pd.DataFrame): A DataFrame containing market data, including the 'Adj Close' column
                           and the buy/sell signal columns specified by `buy_signal_column` and `sell_signal_column`.
    - buy_signal_column (str): The name of the column containing buy signals.
    - sell_signal_column (str): The name of the column containing sell signals.

    Returns:
    - trades (pd.DataFrame): A DataFrame containing details of all trades:
                                ['Entry Date', 'Entry Price', 'Exit Date', 'Exit Price', 'Profit/Loss', 'Profit/Loss (%)']
    """
    open_position = False # track if there is an open position
    trades = []

    for i in range(1, len(data)):
        if data[buy_signal_column].iloc[i] == 1 and not open_position:  # buy 
            open_position = True # open the position
            entry_price = data['Adj Close'].iloc[i]
            entry_date = data.index[i]
            print(f"Buying at {entry_date}: {entry_price:.2f}") 
        elif data[sell_signal_column].iloc[i] == 1 and open_position:  # sell
            exit_price = data['Adj Close'].iloc[i]
            exit_date = data.index[i]
            profit_loss = exit_price - entry_price
            profit_loss_percent = (profit_loss / entry_price) * 100
            trades.append({
                'Entry Date': entry_date,
                'Entry Price': entry_price,
                'Exit Date': exit_date,
                'Exit Price': exit_price,
                'Profit/Loss': profit_loss,
                'Profit/Loss (%)': profit_loss_percent
            })
            print(f"Selling at {exit_date}: {exit_price:.2f} | Profit/Loss: {profit_loss:.2f} ({profit_loss_percent:.2f}%)")
            open_position = False # close the position

    trades = pd.DataFrame(trades)
    return trades


## Win Rate

In [128]:
def calculate_win_rate(trades: pd.DataFrame) -> float:
    """
    Calculates the win rate from the trades DataFrame.

    Parameters:
    - trades (pd.DataFrame): A DataFrame containing trade details from simulate_trading.
    """
    total_trades = len(trades)
    winning_trades = trades['Profit/Loss'][trades['Profit/Loss'] > 0]
    wins = winning_trades.count()

    win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0
    print(f"Win Rate: {win_rate:.2f}%")

## Avg Gain/Loss Ratio

In [129]:
def calculate_avg_gain_loss_ratio(trades: pd.DataFrame) -> float:
    """
    Calculates the Average Gain/Loss Ratio from the trades DataFrame.

    Parameters:
    - trades (pd.DataFrame): A DataFrame containing 'Profit/Loss' for all trades.
    """
    winning_trades = trades['Profit/Loss'][trades['Profit/Loss'] > 0]
    losing_trades = trades['Profit/Loss'][trades['Profit/Loss'] < 0]

    avg_gain = winning_trades.mean() if not winning_trades.empty else 0
    avg_loss = abs(losing_trades.mean()) if not losing_trades.empty else 0

    avg_gain_loss_ratio = avg_gain / avg_loss if avg_loss > 0 else float('inf')

    print(f"Average Gain/Loss Ratio: {avg_gain_loss_ratio:.2f}")

## Maximum Drawdown

In [130]:
def calculate_max_drawdown(data: pd.DataFrame, trades: pd.DataFrame):
    """
        Calculates the Maximum Drawdown of a trading strategy and visualizes it.

        The function computes the maximum drawdown, which is the largest peak-to-trough decline
        in portfolio value, as well as visualizing the stock price, portfolio value, and drawdown
        over time. It assumes a starting investment of $1 and simulates the portfolio value based
        on the given trade entry and exit dates.

        Parameters:
        data (pd.DataFrame): A DataFrame containing the stock's historical price data with column 'Adj Close'.
        trades (pd.DataFrame): A DataFrame containing the trade signals.
    """
    data['Action'] = 0
    data.loc[data.index.isin(trades['Entry Date']), 'Action'] = 1  # Buy (Entry)
    data.loc[data.index.isin(trades['Exit Date']), 'Action'] = -1  # Sell (Exit)

    starting_capital = 1
    portfolio_value = starting_capital
    net_worth = []
    shares_held = 0

    for i in data.index:
        if data.at[i, 'Action'] == 1:  # Buy action
            shares_held = portfolio_value / data.at[i, 'Adj Close']
            portfolio_value = 0
        elif data.at[i, 'Action'] == -1:  # Sell action
            portfolio_value = shares_held * data.at[i, 'Adj Close']
            shares_held = 0
        net_worth.append(portfolio_value + shares_held * data.at[i, 'Adj Close'])

    data['Portfolio Value'] = net_worth

    initial_price = data['Adj Close'].iloc[0]
    data['Stock Growth'] = data['Adj Close'] / initial_price  # Normalize to $1 from the start

    data['Peak Value'] = data['Portfolio Value'].cummax()
    data['Drawdown'] = (data['Portfolio Value'] - data['Peak Value']) / data['Peak Value'] * 100
    max_drawdown = data['Drawdown'].min()

    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        row_heights=[0.5, 0.25, 0.25]
    )

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Adj Close'],
        mode='lines',
        name='Stock Price'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=data.index[data['Action'] == 1],
        y=data['Adj Close'][data['Action'] == 1],
        mode='markers',
        marker=dict(color='green', size=10, symbol='triangle-up'),
        name='Buy Signal'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=data.index[data['Action'] == -1],
        y=data['Adj Close'][data['Action'] == -1],
        mode='markers',
        marker=dict(color='red', size=10, symbol='triangle-down'),
        name='Sell Signal'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Stock Growth'],
        mode='lines',
        name='Buy-and-Hold Strategy',
        line=dict(color='blue')
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Portfolio Value'],
        mode='lines',
        name='Custom Strategy',
        line=dict(color='darkgreen')
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Drawdown'],
        mode='lines',
        name='Drawdown',
        line=dict(color='crimson')
    ), row=3, col=1)

    fig.update_layout(
        title="Stock Price, 1$ Investment Over Time and Drawdown",
        xaxis3_title="Date",
        yaxis_title="Price (USD)",
        yaxis2_title="Value (USD)",
        yaxis3_title="Drawdown (%)",
        legend_title="Legend",
        template="plotly_white",
        height=800
    )

    fig.show()

    print(f"Maximum Drawdown: {max_drawdown:.2f}%")

# Results

In [131]:
#Load data
instrument='HPQ'   # HPQ, AMZN, INTC, DXY, SPY, 
start_date='2019-01-01'
end_date='2024-01-01'
interval='1d'

data = get_data(instrument=instrument, start_date=start_date, end_date=end_date, interval=interval)
data = add_vix(df = data, start_date=start_date, end_date=end_date, interval=interval)

#Calculate strategy
data = macd(data)
data = rsi(data)
data = atr(data)
data = data.dropna()

data = run_strategy(data=data, strategy='Trend-Reversal')

#plot
plot_buy_sell_signal(data=data, instrument=instrument, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal', title = 'Price with Buy/Sell Signals')

trades = simulate_trading(data=data, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal')
calculate_win_rate(trades=trades)
calculate_avg_gain_loss_ratio(trades=trades)
calculate_max_drawdown(data=data, trades=trades)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Buying at 2020-03-25 00:00:00: 13.30
Selling at 2021-02-22 00:00:00: 24.09 | Profit/Loss: 10.80 (81.19%)
Buying at 2022-09-19 00:00:00: 25.13
Win Rate: 100.00%
Average Gain/Loss Ratio: inf


Maximum Drawdown: -21.63%


In [132]:
#Load data
instrument='GS'
start_date='2006-01-01'
end_date='2018-01-01'
interval='1d'

data = get_data(instrument=instrument, start_date=start_date, end_date=end_date, interval=interval)
data = add_vix(df = data, start_date=start_date, end_date=end_date, interval=interval)

#Calculate strategy
data = macd(data)
data = rsi(data)
data = atr(data)
data = data.dropna()

data = run_strategy(data=data, strategy='Trend-Reversal')

#plot
plot_buy_sell_signal(data=data, instrument=instrument, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal', title = 'Price with Buy/Sell Signals')

trades = simulate_trading(data=data, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal')
calculate_win_rate(trades=trades)
calculate_avg_gain_loss_ratio(trades=trades)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Buying at 2006-05-30 00:00:00: 112.34
Selling at 2006-10-20 00:00:00: 135.84 | Profit/Loss: 23.51 (20.93%)
Buying at 2011-05-04 00:00:00: 118.64
Selling at 2012-02-08 00:00:00: 91.81 | Profit/Loss: -26.83 (-22.62%)
Buying at 2012-05-24 00:00:00: 76.80
Selling at 2013-02-08 00:00:00: 121.82 | Profit/Loss: 45.02 (58.62%)
Buying at 2015-08-28 00:00:00: 156.37
Selling at 2016-07-25 00:00:00: 135.28 | Profit/Loss: -21.09 (-13.49%)
Buying at 2017-04-07 00:00:00: 193.91
Win Rate: 50.00%
Average Gain/Loss Ratio: 1.43


In [133]:
#Load data
instrument='SPY'   # HPQ, AMZN, INTC, DXY, SPY, 
start_date='2003-01-01'
end_date='2024-01-01'
interval='1d'

data = get_data(instrument=instrument, start_date=start_date, end_date=end_date, interval=interval)
data = add_vix(df = data, start_date=start_date, end_date=end_date, interval=interval)

#Calculate strategy
data = macd(data)
data = rsi(data)
data = atr(data)
data = data.dropna()

data = run_strategy(data=data, strategy='Mean-Reversal')

#plot
plot_buy_sell_signal(data=data, instrument=instrument, buy_signal_column='Mean Buy Signal', sell_signal_column='Mean Sell Signal', title = 'Price with Buy/Sell Signals')

trades = simulate_trading(data=data, buy_signal_column='Mean Buy Signal', sell_signal_column='Mean Sell Signal')
calculate_win_rate(trades=trades)
calculate_avg_gain_loss_ratio(trades=trades)
calculate_max_drawdown(data=data, trades=trades)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Buying at 2008-10-08 00:00:00: 72.00
Selling at 2012-09-13 00:00:00: 117.34 | Profit/Loss: 45.34 (62.97%)
Buying at 2015-08-24 00:00:00: 161.24
Selling at 2016-12-07 00:00:00: 196.23 | Profit/Loss: 35.00 (21.70%)
Buying at 2018-02-08 00:00:00: 230.82
Selling at 2019-07-03 00:00:00: 275.33 | Profit/Loss: 44.50 (19.28%)
Buying at 2020-02-27 00:00:00: 276.76
Selling at 2023-06-15 00:00:00: 433.55 | Profit/Loss: 156.79 (56.65%)
Win Rate: 100.00%
Average Gain/Loss Ratio: inf


Maximum Drawdown: -32.25%


**Fundamental analysis**

In [134]:
def calculate_historical_pe(tickers, start_date, end_date):
    """
    Fetch historical P/E ratios and combine data for multiple tickers.
    
    Parameters:
    - tickers (list): List of stock symbols (e.g., ['AAPL', 'MSFT']).
    - start_date (str): Start date for the historical data (YYYY-MM-DD).
    - end_date (str): End date for the historical data (YYYY-MM-DD).
    
    Returns:
    - pd.DataFrame: Combined DataFrame with P/E ratios and sector averages.
    """
    dataframes = []

    for ticker in tickers:
        stock_data = yf.Ticker(ticker)

        # Fetch historical prices
        historical_prices = stock_data.history(start=start_date, end=end_date)
        
        # Fetch income statement
        income_statement = stock_data.income_stmt

        if income_statement is not None and not income_statement.empty:
            latest_net_income = income_statement.loc['Net Income'].iloc[0]
            shares_outstanding = stock_data.info.get('sharesOutstanding', None)

            if shares_outstanding and latest_net_income > 0:
                # Calculate the P/E ratio
                pe_ratio = historical_prices['Close'] / (latest_net_income / shares_outstanding)
                historical_prices['P/E Ratio'] = pe_ratio
            else:
                historical_prices['P/E Ratio'] = None
        else:
            historical_prices['P/E Ratio'] = None
        
        # Add ticker for identification
        historical_prices['Ticker'] = ticker

        # Collect relevant data
        dataframes.append(historical_prices[['Close', 'P/E Ratio', 'Ticker']])

    # Combine all stock data
    combined_pe = pd.concat(dataframes)

    return combined_pe

In [135]:
def prepare_pe_data(pe_data):
    """
    Prepare the P/E ratio data by removing duplicates and cleaning up unnecessary columns.
    
    Parameters:
    - pe_data (pd.DataFrame): The raw P/E data output from calculate_historical_pe.

    Returns:
    - pd.DataFrame: The cleaned and prepared P/E data.
    """
    
    if 'Date' in pe_data.columns.duplicated():
        pe_data = pe_data.loc[:, ~pe_data.columns.duplicated()]

    pe_data.reset_index(inplace=True)
    pe_data = pe_data.drop(columns=['level_0', 'index'], errors='ignore')
    pe_data['Date'] = pd.to_datetime(pe_data['Date'])
    pe_data = pe_data[['Date', 'Ticker', 'Close', 'P/E Ratio']]
    pe_data.dropna(subset=['P/E Ratio'], inplace=True)

    return pe_data

In [136]:
def plot_pe_ratios(pe_data):
    """
    Plot the historical P/E ratios for all tickers in the dataset.
    
    Parameters:
    - pe_data (pd.DataFrame): DataFrame containing columns ['Date', 'Ticker', 'P/E Ratio'].
    
    """
    # Ensure the 'Date' column is in datetime format
    pe_data['Date'] = pd.to_datetime(pe_data['Date'])

    # Create a list of unique tickers
    tickers = pe_data['Ticker'].unique()

    # Create a subplot layout with P/E Ratios for all tickers
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Add traces for each company's P/E Ratio
    for ticker in tickers:
        company_data = pe_data[pe_data['Ticker'] == ticker]
        fig.add_trace(go.Scatter(
            x=company_data['Date'],
            y=company_data['P/E Ratio'],
            mode='lines',
            name=f'{ticker} P/E Ratio'
        ))

    # Update layout
    fig.update_layout(
        title="Historical P/E Ratios",
        xaxis_title="Date",
        yaxis_title="P/E Ratio",
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    # Show the plot
    fig.show()

In [137]:
def pe_strategy(pe_data, min_holding_period_days=30):
    """
    Executes the trading strategy based on P/E ratio:
    - Buy: Buy the most undervalued stock (lowest P/E ratio).
    - Sell: Sell if a more undervalued stock appears after the minimum holding period.

    Parameters:
    - pe_data (pd.DataFrame): Prepared data with columns ['Date', 'Ticker', 'Close', 'P/E Ratio'].
    - min_holding_period_days (int): Minimum holding period before selling.

    Returns:
    - pd.DataFrame: Trade details including buy/sell dates, prices, and profit/loss.
    """
    trades = []  # List to store trade details
    open_position = None  # Track the current open position (if any)
    min_holding_period = pd.Timedelta(days=min_holding_period_days)

    # Loop through each unique date in the data
    for date in pe_data['Date'].unique():
        # Filter data for the current date
        date_data = pe_data[pe_data['Date'] == date]

        # Check if a position is currently held
        if open_position is None:
            # Look for undervalued stocks to buy
            candidates = date_data.sort_values(by='P/E Ratio')
            if not candidates.empty:
                # Buy the most undervalued stock
                stock_to_buy = candidates.iloc[0]
                open_position = {
                    'Ticker': stock_to_buy['Ticker'],
                    'Date_Buy': stock_to_buy['Date'],
                    'Price_Buy': stock_to_buy['Close'],
                    'P/E Ratio_Buy': stock_to_buy['P/E Ratio']
                }
                print(f"Buying {stock_to_buy['Ticker']} on {stock_to_buy['Date']}")
        else:
            # Check the held stock's P/E ratio
            held_stock = date_data[date_data['Ticker'] == open_position['Ticker']]
            if not held_stock.empty:
                # Check if minimum holding period has passed
                buy_date = pd.to_datetime(open_position['Date_Buy'])
                current_date = pd.to_datetime(held_stock['Date'].iloc[0])
                holding_period = current_date - buy_date

                # Check if a more undervalued stock exists
                more_undervalued = date_data[date_data['P/E Ratio'] < 0.9 * held_stock['P/E Ratio'].iloc[0]]

                # Sell if a more undervalued stock exists and holding period is satisfied
                if not more_undervalued.empty and holding_period >= min_holding_period:
                    open_position.update({
                        'Date_Sell': held_stock['Date'].iloc[0],
                        'Price_Sell': held_stock['Close'].iloc[0],
                        'Profit/Loss': held_stock['Close'].iloc[0] - open_position['Price_Buy'],
                        'Profit/Loss (%)': ((held_stock['Close'].iloc[0] - open_position['Price_Buy']) / open_position['Price_Buy']) * 100
                    })
                    trades.append(open_position)
                    print(f"Selling {open_position['Ticker']} on {open_position['Date_Sell']} with P/L: {open_position['Profit/Loss']:.2f}")
                    open_position = None  # Reset position for the next trade

    # Handle any open position at the end of the data
    if open_position is not None:
     last_date = pe_data['Date'].max()
     last_price = pe_data[(pe_data['Ticker'] == open_position['Ticker']) & (pe_data['Date'] == last_date)]['Close'].iloc[0]
     open_position.update({
        'Date_Sell': last_date,
        'Price_Sell': last_price,
        'Profit/Loss': last_price - open_position['Price_Buy'],
        'Profit/Loss (%)': ((last_price - open_position['Price_Buy']) / open_position['Price_Buy']) * 100,
        'Unrealized': True  # Mark it as an unrealized trade
    })
     
    trades.append(open_position)
    print(f"Unrealized trade for {open_position['Ticker']} on {last_date} with P/L: {open_position['Profit/Loss']:.2f}")


    # Convert trades to a DataFrame for analysis
    trades_df = pd.DataFrame(trades)

    return trades_df


In [138]:
def calculate_performance(trades_df):
    """
    Calculate and display performance metrics for the trading strategy.
    
    Parameters:
    - trades_df (pd.DataFrame): DataFrame containing trade details with columns 
                                ['Profit/Loss', 'Profit/Loss (%)'].
                                
    Returns:
    - dict: A dictionary with all performance metrics.
    """
    
    metrics = {}

    # 1. Total Profit/Loss
    metrics['Total Profit/Loss'] = trades_df['Profit/Loss'].sum()

    # 2. Total Profit/Loss Percentage
    metrics['Total Profit/Loss (%)'] = trades_df['Profit/Loss (%)'].sum()

    # 3. Win Rate
    total_trades = len(trades_df)
    winning_trades = trades_df[trades_df['Profit/Loss'] > 0]
    metrics['Win Rate (%)'] = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0

    # 4. Average Profit/Loss
    metrics['Average Profit/Loss'] = trades_df['Profit/Loss'].mean() if total_trades > 0 else 0

    # 5. Average Gain
    metrics['Average Gain'] = winning_trades['Profit/Loss'].mean() if not winning_trades.empty else 0

    # 6. Average Loss
    losing_trades = trades_df[trades_df['Profit/Loss'] < 0]
    metrics['Average Loss'] = abs(losing_trades['Profit/Loss'].mean()) if not losing_trades.empty else 0

    # 7. Gain/Loss Ratio
    metrics['Gain/Loss Ratio'] = (
        metrics['Average Gain'] / metrics['Average Loss']
        if metrics['Average Loss'] > 0 else float('inf')
    )

    # 8. Total Trades
    metrics['Total Trades'] = total_trades

    # Print all metrics for convenience
    for key, value in metrics.items():
        print(f"{key}: {value:.2f}" if isinstance(value, (int, float)) else f"{key}: {value}")
    
    return metrics

In [139]:
# Parameters
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA"]  

start_date = "2018-01-01"  
end_date = "2023-01-01"  
min_holding_period_days = 30  # minimum holding period before selling


pe_data = calculate_historical_pe(tickers, start_date, end_date)
pe_data = prepare_pe_data(pe_data)
plot_pe_ratios(pe_data)

# trades
trades = pe_strategy(pe_data, min_holding_period_days)
# print("\nTrade Results:")
# print(trades)

#performance metrics
print("-----------------------------------------------------------------------------------------")
metrics = calculate_performance(trades)
print(metrics)

C:\Users\Kateryna\AppData\Local\Temp\ipykernel_16892\1500870743.py:12: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Buying NVDA on 2018-01-02 00:00:00-05:00
Selling NVDA on 2018-02-07 00:00:00-05:00 with P/L: 0.73
Buying GOOGL on 2018-02-08 00:00:00-05:00
Selling GOOGL on 2018-07-24 00:00:00-04:00 with P/L: 12.49
Buying TSLA on 2018-07-25 00:00:00-04:00
Selling TSLA on 2018-10-26 00:00:00-04:00 with P/L: 1.48
Buying NVDA on 2018-10-29 00:00:00-04:00
Selling NVDA on 2019-05-22 00:00:00-04:00 with P/L: -0.82
Buying TSLA on 2019-05-23 00:00:00-04:00
Selling TSLA on 2019-10-25 00:00:00-04:00 with P/L: 8.84
Buying NVDA on 2019-10-28 00:00:00-04:00
Selling NVDA on 2020-03-24 00:00:00-04:00 with P/L: 1.06
Buying GOOGL on 2020-03-25 00:00:00-04:00
Unrealized trade for GOOGL on 2022-12-30 00:00:00-05:00 with P/L: 33.07
-----------------------------------------------------------------------------------------
Total Profit/Loss: 56.85
Total Profit/Loss (%): 177.66
Win Rate (%): 85.71
Average Profit/Loss: 8.12
Average Gain: 9.61
Average Loss: 0.82
Gain/Loss Ratio: 11.72
Total Trades: 7.00
{'Total Profit/Loss': 5

In [140]:
# Parameters
tickers = [
    "NEP",  # NextEra Energy Partners
    "BEP",  # Brookfield Renewable Partners
    "PLUG", # Plug Power Inc.
    "ENPH", # Enphase Energy
    "RUN",  # Sunrun Inc.
    "SEDG", # SolarEdge Technologies
    "ORA",  # Ormat Technologies
    "FSLR"  # First Solar Inc.
 ] # Energy sector


start_date = "2014-01-01"  
end_date = "2020-01-01"  
min_holding_period_days = 30  # minimum holding period before selling


pe_data = calculate_historical_pe(tickers, start_date, end_date)
pe_data = prepare_pe_data(pe_data)
plot_pe_ratios(pe_data)

# trades
trades = pe_strategy(pe_data, min_holding_period_days)
# print("\nTrade Results:")
# print(trades)

#performance metrics
print("-----------------------------------------------------------------------------------------")
metrics = calculate_performance(trades)
print(metrics)

C:\Users\Kateryna\AppData\Local\Temp\ipykernel_16892\1500870743.py:12: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Buying ENPH on 2014-06-27 00:00:00-04:00
Selling ENPH on 2019-08-07 00:00:00-04:00 with P/L: 22.30
Buying FSLR on 2019-08-08 00:00:00-04:00
Selling FSLR on 2019-09-10 00:00:00-04:00 with P/L: -2.38
Buying ENPH on 2019-09-11 00:00:00-04:00
Selling ENPH on 2019-10-25 00:00:00-04:00 with P/L: -0.17
Buying FSLR on 2019-10-28 00:00:00-04:00
Unrealized trade for FSLR on 2019-12-31 00:00:00-05:00 with P/L: 2.72
-----------------------------------------------------------------------------------------
Total Profit/Loss: 22.47
Total Profit/Loss (%): 263.41
Win Rate (%): 50.00
Average Profit/Loss: 5.62
Average Gain: 12.51
Average Loss: 1.28
Gain/Loss Ratio: 9.81
Total Trades: 4.00
{'Total Profit/Loss': 22.46999740600586, 'Total Profit/Loss (%)': 263.4080992022218, 'Win Rate (%)': 50.0, 'Average Profit/Loss': 5.617499351501465, 'Average Gain': 12.50999927520752, 'Average Loss': 1.2750005722045898, 'Gain/Loss Ratio': 9.811759734018482, 'Total Trades': 4}


In [141]:
# Parameters
tickers = ["PG", "UL", "NSRGY", "KO", "PEP", "LRLCY", "CL", "KMB", "RBGLY"] # consumer goods

start_date = "2018-01-01"  
end_date = "2023-01-01"  
min_holding_period_days = 30  # minimum holding period before selling


pe_data = calculate_historical_pe(tickers, start_date, end_date)
pe_data = prepare_pe_data(pe_data)
plot_pe_ratios(pe_data)

# trades
trades = pe_strategy(pe_data, min_holding_period_days)
# print("\nTrade Results:")
# print(trades)

#performance metrics
print("-----------------------------------------------------------------------------------------")
metrics = calculate_performance(trades)
print(metrics)

C:\Users\Kateryna\AppData\Local\Temp\ipykernel_16892\1500870743.py:12: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



Buying PG on 2018-01-02 00:00:00-05:00
Selling PG on 2020-07-09 00:00:00-04:00 with P/L: 34.35
Buying KO on 2020-07-10 00:00:00-04:00
Selling KO on 2021-10-08 00:00:00-04:00 with P/L: 9.78
Buying UL on 2021-10-11 00:00:00-04:00
Unrealized trade for UL on 2022-12-30 00:00:00-05:00 with P/L: 0.23
-----------------------------------------------------------------------------------------
Total Profit/Loss: 44.36
Total Profit/Loss (%): 70.78
Win Rate (%): 100.00
Average Profit/Loss: 14.79
Average Gain: 14.79
Average Loss: 0.00
Gain/Loss Ratio: inf
Total Trades: 3.00
{'Total Profit/Loss': 44.359718322753906, 'Total Profit/Loss (%)': 70.78184245088501, 'Win Rate (%)': 100.0, 'Average Profit/Loss': 14.786572774251303, 'Average Gain': 14.786572774251303, 'Average Loss': 0, 'Gain/Loss Ratio': inf, 'Total Trades': 3}
